In [5]:
from src.datasets import ImageDataset
from glob import glob
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
from results import metrics
import numpy as np
import pandas as pd
from scipy import stats
from statannotations.Annotator import Annotator
import sys 
import importlib
import torch
import os
import nibabel as nib
from results import denorm
from nilearn import plotting 
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps
import matplotlib.pyplot as plt
import random

In [7]:
expe_list = ['2class', '2class-30grp', '2class-spm', '2class-30grp-spm', '4class', '4class-30grp']

for expe in expe_list: 
    p_source_list = np.unique(
        [f.split('_')[-2][5:] for f in sorted(
            glob(
                f'/Volumes/egermani/stargan/samples-{expe}/trg_img-*_orig-*_target-*.nii.gz'
            )
        )
        ]
    )
    p_target_list = np.unique(
        [f.split('_')[-1].split('.')[0][7:] for f in sorted(
            glob(
                f'/Volumes/egermani/stargan/samples-{expe}/trg_img-*_orig-*_target-*.nii.gz'
            )
        )
        ]
    )

    for p_source in p_source_list:
        for p_target in p_target_list:
        
            data_trg = sorted(
                glob(
                    f'/Volumes/egermani/stargan/samples-{expe}/trg_img-*_orig-{p_source}_target-{p_target}.nii.gz'
                )
            )
            trg_dataset = ImageDataset(data_trg)
            trg_loader = DataLoader(
                trg_dataset, 
                batch_size=1,
                shuffle=False)
            
            data_gen = sorted(
                glob(
                    f'/Volumes/egermani/stargan/samples-{expe}/gen_img-*_orig-{p_source}_target-{p_target}.nii.gz'
                )
            )
            gen_dataset = ImageDataset(data_gen)
            gen_loader = DataLoader(
                gen_dataset, 
                batch_size=1,
                shuffle=False)
            
            data_src = sorted(
                glob(
                    f'/Volumes/egermani/stargan/samples-{expe}/src_img-*_orig-{p_source}_target-{p_target}.nii.gz'
                )
            )
            src_dataset = ImageDataset(data_src)
            src_loader = DataLoader(
                src_dataset, 
                batch_size=1,
                shuffle=False)
            
            # Source - Target correlations
            corr_s_t = []
            
            for data1, data2 in zip(trg_loader, src_loader):
                corr_s_t.append(
                    metrics.get_correlation(
                        data1.detach().numpy(), 
                        data2.detach().numpy()
                    )
                )
            
            # Source - Target correlations
            corr_g_t = []
            
            for data1, data2 in zip(trg_loader, gen_loader):
                corr_g_t.append(
                    metrics.get_correlation(
                        data1.detach().numpy(), 
                        data2.detach().numpy()
                    )
                )
            
            df_st = pd.DataFrame(
                {'Type' :['Source-Target' for i in range(len(corr_s_t))],
                 'Correlation' : corr_s_t}
            )
            
            df_tg = pd.DataFrame(
                {'Type' :['Target-Generated' for i in range(len(corr_g_t))],
                 'Correlation' : corr_g_t}
            )
            
            df = pd.concat([df_st, df_tg])
            
            df.to_csv(f'./figures/correlations-{expe}-{p_source}-{p_target}.csv')

In [6]:
df_mean.to_csv('mean_correlations-2classes.csv')

In [8]:
df_mean

,orig-target,orig-gen,gen-target
transfer,,,
spm-5-0-0_spm-5-0-0,1.000000,0.867288,0.867288
spm-5-0-0_spm-5-0-1,0.928566,0.780008,0.819315
spm-5-0-1_spm-5-0-0,0.928566,0.783352,0.869394
spm-5-0-1_spm-5-0-1,1.000000,0.820946,0.820946
